In [21]:
RAWG_API_KEY_1 = "96e86102c0c84a729c42df1af5810868"
RAWG_API_KEY_2 = "146c248d972940b68958e61a6109de21"

In [ ]:
import json
import pandas as pd

with open(r"..\assets\RAWG\games_data.json", 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.shape

(880640, 30)

In [9]:
df_steam = df[df['stores'].apply(lambda stores: any(store['store']['id'] == 1 for store in stores))]
df_steam.shape

(107324, 30)

In [15]:
df_steam_1 = df_steam.iloc[0:30000].copy()
df_steam_2 = df_steam.iloc[30000:60000].copy()
df_steam_3 = df_steam.iloc[60000:90000].copy()
df_steam_4 = df_steam.iloc[90000:-1].copy()

In [ ]:
# request API to get the steam store link, then extract the steam app id from this link

import aiohttp
import asyncio
import re
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio

async def get_steam_id(session, rawg_id, api_key):
    url = f"https://api.rawg.io/api/games/{rawg_id}/stores?key={api_key}"
    async with session.get(url) as response:
        data = await response.json()
        for store in data.get('results', []):
            if store['store_id'] == 1:  # Steam
                steam_url = store['url']
                match = re.search(r'store\.steampowered\.com\/app\/(\d+)', steam_url)
                if match:
                    return rawg_id, match.group(1)
    return rawg_id, None

async def fetch_all_steam_ids(df_input, api_key, max_connections=100):
    df = df_input.copy()

    connector = aiohttp.TCPConnector(limit=max_connections)
    steam_ids = {}

    async with aiohttp.ClientSession(connector=connector) as session:
        tasks = [get_steam_id(session, rawg_id, api_key) for rawg_id in df['id'].tolist()]
        
        for future in tqdm_asyncio.as_completed(tasks, total=len(tasks)):
            result = await future
            rawg_id, steam_id = result
            steam_ids[rawg_id] = steam_id

    # Map Steam IDs back to dataframe
    df['steam_app_id'] = df['id'].map(steam_ids)
    return df

nest_asyncio.apply()
df_steam_4_id = asyncio.run(fetch_all_steam_ids(df_steam_4, RAWG_API_KEY_2))

100%|██████████| 17323/17323 [01:43<00:00, 166.72it/s]


In [ ]:
df_steam_1_id[df_steam_1_id['steam_app_id'].notnull()][['name', 'steam_app_id']]

,name,steam_app_id
0,Grand Theft Auto V,3240220
1,The Witcher 3: Wild Hunt,292030
2,Portal 2,620
3,Counter-Strike: Global Offensive,730
4,Tomb Raider (2013),203160
...,...,...
43785,Fantasia of the Wind 2 风之幻想曲 第二部,1006400
43786,枕边少女 MOE Hypnotist - share dreams with you,942930
43787,The Den of Chaos - 混沌の魔窟殿～アヒアハン１９世の指令編～,1007280
43789,Swordbreaker: Back to The Castle,1005990


In [25]:
df_steam_2_id[df_steam_2_id['steam_app_id'].notnull()][['name', 'steam_app_id']]

,name,steam_app_id
43816,Poopy Philosophy,821120
43837,Umihara Kawase Fresh!,1272680
43848,Super Ledgehop: Double Laser,787070
43891,Cosmoteer,799600
43917,Achievement Hunter: Zombie 2,790400
...,...,...
116724,Woohoo!,2976020
116728,Sunset afterglow,3061590
116729,Magic of Runes: Hidden Object Game,2712750
116732,Radioactive Pineapple,3058210


In [27]:
df_steam_3_id[df_steam_3_id['steam_app_id'].notnull()][['name', 'steam_app_id']]

,name,steam_app_id
118620,ShitMan,2741930
118621,Abyssal Descent,2506740
118622,愤怒的孙子们,2930120
118623,Breed Kim Il-Sung,2915070
118624,Click on bells together,3005850
...,...,...
452413,Monster Wilds,1691510
452561,Amazething,1443600
452655,Green Garden,2204710
452799,Nekomancy,2093760


In [29]:
df_steam_4_id[df_steam_4_id['steam_app_id'].notnull()][['name', 'steam_app_id']]

,name,steam_app_id
462190,Infinity ZigZag,1596060
462193,SLASH OF BULLET,1559790
462196,Flip-Out!,962310
506761,Vaccine19,1315070
506762,MUMMMASTER!,1489480
506763,Fast Cubes,1490910
506767,Ayu - The Dream Soul,1446660
506771,Christmas Hentai,1495840
533123,Next Earth: The Journey Trilogy,1353660
545594,Sudoku Dreams,1345430


In [30]:
df_steam_id = pd.concat([df_steam_1_id, df_steam_2_id, df_steam_3_id, df_steam_4_id]).dropna(subset=['steam_app_id'])
df_steam_id.shape

(62138, 31)

In [31]:
df_steam_id.sample(5)

,id,slug,name,released,tba,background_image,rating,rating_top,ratings,ratings_count,...,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating,steam_app_id
9289,19090,vinyl,Vinyl,2016-07-01,False,https://media.rawg.io/media/screenshots/15c/15...,0.00,0,"[{'id': 1, 'title': 'skip', 'count': 3, 'perce...",3,...,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 21348, 'store': {'id': 1, 'name': 'Ste...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0,369030
8267,20621,burstfire,Burstfire,2015-09-24,False,https://media.rawg.io/media/screenshots/7dd/7d...,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 2, 'percen...",4,...,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 23193, 'store': {'id': 1, 'name': 'Ste...",None,"[{'id': 7, 'name': 'Multiplayer', 'slug': 'mul...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0,349580
451838,600946,booble-hentai,Booble Hentai,2021-05-07,False,https://media.rawg.io/media/screenshots/565/56...,0.00,0,[],0,...,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 595230, 'store': {'id': 1, 'name': 'St...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0,1566920
4631,14886,survive-in-space,Survive in Space,2016-05-20,False,https://media.rawg.io/media/screenshots/29c/29...,2.05,1,"[{'id': 1, 'title': 'skip', 'count': 13, 'perc...",21,...,"[{'platform': {'id': 5, 'name': 'macOS', 'slug...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 16393, 'store': {'id': 1, 'name': 'Ste...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN,467950
379269,684717,dungeon-of-death,Dungeon of Death,2021-10-22,False,https://media.rawg.io/media/screenshots/4e3/4e...,0.00,0,[],0,...,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 887889, 'store': {'id': 1, 'name': 'St...",None,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",None,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0,2205160


In [ ]:
import joblib
joblib.dump(df_steam_id, r'..\assets\RAWG\games_data_with_id.pkl')

['games_data_with_id.pkl']